## Section 0


### Section 0.1

First, lets load the libraries we need

In [1]:
import os
import re
import html
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from matching_helpers import normaliser,\
                             parse_datetime,\
                             read_raw_data,\
                             prepare_nhsspend,\
                             prepare_contractsfinder,\
                             org_counter,\
                             strip_html,\
                             unique_agg,\
                             process_dates,\
                             make_matches

### Section 0.2

Lets load the raw payments data we need

In [2]:
tqdm.pandas()
df_centgov = read_raw_data('centgov_data.csv', 'Contracts Finder')
df_nhs = read_raw_data('nhsspend_data.csv', 'NHSSpend')
df_contracts = read_raw_data('contractsfinder_data.csv', 'Contracts Finder')

### Section 0.3

Wrangle them a bit

In [3]:
df_centgov = df_centgov[['data_source', 'amount', 'supplier', 'date', 'dept']]

In [4]:
df_nhs = prepare_nhsspend(df_nhs)

In [5]:
df_contracts = prepare_contractsfinder(df_contracts)

### Section 0.4

Merge and clean them a bit

In [6]:
df_comb = pd.concat([df_nhs, df_centgov, df_contracts], ignore_index=True)
df_comb = df_comb.rename({'supplier': 'SUPPLIER'}, axis=1)
df_comb['SUPPLIER'] = df_comb['SUPPLIER'].str.upper().str.strip()

df_comb['SUPPLIER_NUMERIC'] = pd.to_numeric(df_comb['SUPPLIER'], errors='coerce')
print(f'Dropping {len(df_comb[df_comb["SUPPLIER"].isnull()])} rows of data because of numeric suppliers')
df_comb = df_comb[df_comb['SUPPLIER'].notnull()]
df_comb = df_comb[df_comb['SUPPLIER_NUMERIC'].isna()]
df_comb = df_comb.drop(columns='SUPPLIER_NUMERIC')

print(f'Dropping {len(df_comb[df_comb["SUPPLIER"].isnull()])} rows of data because of NaN suppliers')
df_comb = df_comb[df_comb['SUPPLIER'].notnull()]
df_comb['SUPPLIER'] = df_comb['SUPPLIER'].progress_apply(strip_html)
print(f'Dropping {len(df_comb[df_comb["SUPPLIER"].isnull()])} rows of data after html parsing')
df_comb = df_comb[df_comb['SUPPLIER'].notnull()]
df_comb['date'] = df_comb['date'].astype(str).str.split('T').str[0]
df_comb['date'] = pd.to_datetime(df_comb['date'],
                                 format='mixed',
                                 errors='coerce')
df_comb['date'] = df_comb['date'].map(lambda x: x.strftime('%d-%m-%Y') if pd.notnull(x) else np.nan)
print(f'Dropping {len(df_comb[df_comb["date"].isnull()])} rows of data due to NaN dates')
df_comb = df_comb[df_comb['date'].notnull()]
print(f'Dropping {len(df_comb[df_comb["amount"].isnull()])} rows of data due to NaN amounts')
df_comb = df_comb[df_comb['amount'].notnull()]
print(f'Dropping {len(df_comb[df_comb["dept"].isnull()])} rows of data due to NaN depts')
df_comb = df_comb[df_comb['dept'].notnull()]
df_comb['NORMALIZED_SUPPLIER'] = df_comb['SUPPLIER'].progress_apply(normaliser)
df_comb = df_comb[df_comb['NORMALIZED_SUPPLIER'].apply(isinstance, args=(str,))]

rows_to_drop = len(df_comb[
    (df_comb["SUPPLIER"].str.len() <= 3) |
    (df_comb["NORMALIZED_SUPPLIER"].str.len() <= 3)
])

# Print the message with the count of rows to be dropped
print(f'Dropping {rows_to_drop} rows of data due to supplier str len<=3')

df_comb = df_comb[
    (df_comb["SUPPLIER"].str.len() > 3) |
    (df_comb["NORMALIZED_SUPPLIER"].str.len() > 3)
]

df_comb[['SUPPLIER', 'ORG_COUNT']] = df_comb['SUPPLIER'].apply(lambda x: pd.Series(org_counter(x)))
all_rows = len(df_comb)

for supplier in ["SUCCESSFUL SUPPL",
                 "SEE ATTACH",
                 "REFER ATTACH",
                 "CONTRACT WAS AWARD",
                 "AWARDED SUPPLIERS",
                 "SUCCESSFUL SUPPLIER",
                 "PLEASE SEE",
                 'NAMED IND',
                 'REDACT',
                 "PLEASE REFER"]:
    df_comb = df_comb[~df_comb['SUPPLIER'].str.contains(supplier)]

print(f'Number of rows dropped due to redacted: {len(df_comb)-all_rows}')

print(f'Dropping {len(df_comb[df_comb["ORG_COUNT"]!=1])} where org_count !=1')
df_comb = df_comb[df_comb['ORG_COUNT']==1]

Dropping 926 rows of data because of numeric suppliers
Dropping 0 rows of data because of NaN suppliers


  0%|          | 0/9331286 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Section 0.5

Merge them into a unique dataframe, and then process it (inc. normalize)

In [ ]:
df_uniq = df_comb.pivot_table(index=['SUPPLIER'],
                              values=['date',
                                      'contractsfinder_region',
                                      'contractsfinder_awardedToVcse',
                                      'dept'],
                              aggfunc=unique_agg).reset_index()
df_sum = df_comb.groupby('SUPPLIER')['amount'].sum().reset_index()
df_counts = df_comb['SUPPLIER'].value_counts().reset_index()
df_uniq = pd.merge(df_uniq,
                   df_sum,
                   how='left',
                   left_on='SUPPLIER',
                   right_on='SUPPLIER'
                  )
df_uniq[['SUPPLIER', 'ORG_COUNT']] = df_uniq['SUPPLIER'].progress_apply(lambda x: pd.Series(org_counter(x)))
df_uniq['NORMALIZED_SUPPLIER'] = df_uniq['SUPPLIER'].progress_apply(normaliser)
df_uniq = df_uniq[df_uniq['NORMALIZED_SUPPLIER'].apply(isinstance, args=(str,))]
df_uniq = pd.merge(df_uniq,
                   df_counts,
                   how='left',
                   left_on='SUPPLIER',
                   right_on='SUPPLIER'
                  )
df_uniq.sort_values(by=['amount'],
                    ascending=False)

df_uniq1 = df_nhs[['supplier',
                   'NHSSpend_CompanyName',
                   'NHSSpend_CompanyNumber',
                   'NHSSpend_CharityName',
                   'NHSSpend_CharityRegNo',
                   'NHSSpend_CharitySubNo',
                   'NHSSpend_CharityNameNo',
                   'NHSSpend_CharityName']].drop_duplicates()
df_uniq = pd.merge(df_uniq,
                   df_uniq1,
                   how='left',
                   left_on='SUPPLIER',
                   right_on='supplier'
                  )


contractsfinder_region = df_sum = df_comb.groupby('SUPPLIER')['amount'].sum().reset_index()


df_uniq = df_uniq.rename({'count': 'PAYMENT_TOTAL_COUNT',
                          'amount': 'PAYMENT_TOTAL_AMOUNT'},
                         axis=1)
print(f'Dropping {len(df_uniq[df_uniq["ORG_COUNT"]!=1])} org_count !=1')
df_uniq = df_uniq[df_uniq['ORG_COUNT']==1]
df_uniq = df_uniq.drop(columns='supplier')
df_uniq['contractsfinder_awardedToVcse'] = df_uniq['contractsfinder_awardedToVcse'].apply(lambda x: "True" if True in x else "False")
df_uniq['deptcount'] = df_uniq['dept'].apply(len)
df_uniq['contractsfinder_region'] = df_uniq['contractsfinder_region'].apply(lambda x: "" if x == [np.nan] else x)
df_uniq['date'] = df_uniq['date'].astype(str).progress_apply(process_dates)
df_uniq['SUPPLIER'] = df_uniq['SUPPLIER'].replace('"', "[DQ]", regex=True)
df_uniq['dept'] = df_uniq['dept'].replace('"', "[DQ]", regex=True)
df_comb['SUPPLIER'] = df_comb['SUPPLIER'].replace('"', "[DQ]", regex=True)

### Section 0.6

Drop stuff that isn't any longer needed

In [ ]:
df_uniq = df_uniq.drop('NHSSpend_CharityNameNo', axis=1)
df_uniq = df_uniq.drop('NHSSpend_CharityName', axis=1)
df_uniq = df_uniq.drop('NHSSpend_CompanyName', axis=1)

df_comb = df_comb.drop('NHSSpend_CharityNameNo', axis=1)
df_comb = df_comb.drop('NHSSpend_CharityName', axis=1)
df_comb = df_comb.drop('NHSSpend_CompanyName', axis=1)
df_comb = df_comb.drop('NHSSpend_audit_type', axis=1)
df_comb = df_comb.drop('NHSSpend_CHnotes', axis=1)
df_comb = df_comb.drop('NHSSpend_CCnotes', axis=1)

### Section 0.7

See what's left in our dataframes

In [9]:
print(df_uniq.columns)
print(df_uniq.head())

Index(['SUPPLIER', 'contractsfinder_awardedToVcse', 'contractsfinder_region',
       'date', 'dept', 'PAYMENT_TOTAL_AMOUNT', 'ORG_COUNT',
       'NORMALIZED_SUPPLIER', 'PAYMENT_TOTAL_COUNT', 'NHSSpend_CompanyNumber',
       'NHSSpend_CharityRegNo', 'NHSSpend_CharitySubNo', 'deptcount'],
      dtype='object')
                                 SUPPLIER contractsfinder_awardedToVcse  \
0             KPMG ADVISORY LIMITED                         False   
1                  [DQ] INVESTIGO LTD[DQ]                         False   
2   [DQ] PRICEWATERHOUSECOOPERS (PWC)[DQ]                         False   
3                    [DQ] SOFTCAT LTD[DQ]                         False   
4  [DQ] TRANSOFT SOLUTIONS UK LIMITED[DQ]                         False   

                              contractsfinder_region        date  \
0                                       [Any region]  20-07-2018   
1  [North East,North West,Yorkshire and The Humbe...  08-04-2022   
2  [North East,North West,Yorkshire

In [10]:
print(df_comb.columns)
print(df_comb.head())

Index(['data_source', 'amount', 'SUPPLIER', 'date', 'dept',
       'NHSSpend_CompanyNumber', 'NHSSpend_CharityRegNo',
       'NHSSpend_CharitySubNo', 'NHSSpend_isCIC',
       'contractsfinder_awardedToVcse', 'contractsfinder_region',
       'NORMALIZED_SUPPLIER', 'ORG_COUNT'],
      dtype='object')
  data_source    amount                                       SUPPLIER  \
0    NHSSpend   69366.0                     BMI HEALTHCARE COLLECTIONS   
1    NHSSpend   43854.0                 NORTH YORKSHIRE COUNTY COUNCIL   
2    NHSSpend   32278.0                           NOTTINGHAM REHAB LTD   
3    NHSSpend   48127.0             LEEDS TEACHING HOSPITALS NHS TRUST   
4    NHSSpend  215687.0  HARROGATE & DISTRICT NHS FOUNDATION NHS TRUST   

         date          dept NHSSpend_CompanyNumber  NHSSpend_CharityRegNo  \
0  30-09-2017  NHS_HRAW_CCG               02164270                    NaN   
1  30-09-2017  NHS_HRAW_CCG                    NaN                    NaN   
2  30-09-2017  NHS_HRAW_

### Section 0.8

Sort, coerce, save uniq out before normalizing

In [ ]:
df_uniq = df_uniq.sort_values(by='PAYMENT_TOTAL_AMOUNT',
                              ascending=False)
print(len(df_uniq))
df_uniq = df_uniq.copy()[df_uniq['NORMALIZED_SUPPLIER'].notnull()]
print(len(df_uniq))
df_uniq = df_uniq[df_uniq['SUPPLIER'].notnull()]
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_raw.csv'),
               index=False)

In [ ]:
df_comb.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_all_raw.csv'),
               index=False
              )

# Section 1.0 

Lets now process the other auxillery registers

### Section 1.1

Lets load the uniq file back in so that we can split up the cleaning and matching process if we want to

### Section 1.1

CH first:

In [13]:
df_ch = pd.read_csv(os.path.join(
    '..', 'registers', 'BasicCompanyDataAsOneFile-2024-08-01.csv'),
                    usecols=['CompanyName', ' CompanyNumber', 'RegAddress.PostTown', 'RegAddress.PostCode']
                )
print('The length of the most recent CH file is ', len(df_ch))

df_ch_old_1 = pd.read_csv(os.path.join(
    '..', 'registers', 'BasicCompanyData-2012-07-01-part1_4.csv'),
                          usecols=['CompanyName', ' CompanyNumber', 'RegAddress.PostTown', 'RegAddress.PostCode']
                         )
print('The length of part 1 of the older CH file is ', len(df_ch_old_1))

df_ch_old_2 = pd.read_csv(os.path.join(
    '..', 'registers', 'BasicCompanyData-2012-07-01-part2_4.csv'),
                          usecols=['CompanyName', ' CompanyNumber', 'RegAddress.PostTown', 'RegAddress.PostCode']
                         )
print('The length of part 2 of the older CH file is ', len(df_ch_old_2))

df_ch_old_3 = pd.read_csv(os.path.join(
    '..', 'registers', 'BasicCompanyData-2012-07-01-part3_4.csv'),
                          usecols=['CompanyName', ' CompanyNumber', 'RegAddress.PostTown', 'RegAddress.PostCode']
                         )
print('The length of part 3 of the older CH file is ', len(df_ch_old_3))

df_ch_old_4 = pd.read_csv(os.path.join(
    '..', 'registers', 'BasicCompanyData-2012-07-01-part4_4.csv'),
                          usecols=['CompanyName', ' CompanyNumber', 'RegAddress.PostTown', 'RegAddress.PostCode']
                         )
print('The length of part 4 of the older CH file is ', len(df_ch_old_4))

df_ch = pd.concat([df_ch,df_ch_old_1], axis=0, ignore_index=True)
df_ch = pd.concat([df_ch,df_ch_old_2], axis=0, ignore_index=True)
df_ch = pd.concat([df_ch,df_ch_old_3], axis=0, ignore_index=True)
df_ch = pd.concat([df_ch,df_ch_old_4], axis=0, ignore_index=True)

df_ch = df_ch.drop_duplicates(subset=[' CompanyNumber'])
print('The length of the merged CH file is ', len(df_ch))

The length of the most recent CH file is  5621958
The length of part 1 of the older CH file is  849999
The length of part 2 of the older CH file is  850000
The length of part 3 of the older CH file is  850000
The length of part 4 of the older CH file is  445775
The length of the merged CH file is  7324319


In [14]:
df_ch['NORMALIZED_CompanyName'] = df_ch['CompanyName'].astype(str).progress_apply(normaliser)
df_ch = df_ch[df_ch['NORMALIZED_CompanyName'].apply(isinstance, args=(str,))]

df_ch = df_ch[df_ch['NORMALIZED_CompanyName'].notnull()]
df_ch = df_ch.drop_duplicates(subset=['NORMALIZED_CompanyName'], keep=False)
df_ch = df_ch[[' CompanyNumber', 'NORMALIZED_CompanyName', 'CompanyName', 'RegAddress.PostTown', 'RegAddress.PostCode']]
df_ch['CompanyName'] = df_ch['CompanyName'].replace('"', "[DQ]", regex=True)
df_ch.to_csv(os.path.join('..', 'registers', 'ch_w_normalised.csv'), index=False)

  0%|          | 0/7324319 [00:00<?, ?it/s]

### Section 1.2

Now the spine:

In [15]:
df_spine = pd.read_csv(os.path.join(
    '..', 'registers', 'public_spine.spine.csv'),
                       usecols=['uid', 'organisationname', 'fulladdress', 'city', 'postcode', 'registerdate', 'removeddate'],
                       low_memory=False,
                )
df_spine['NORMALIZED_organisationname'] = df_spine['organisationname'].astype(str).progress_apply(normaliser)
df_spine = df_spine[df_spine['NORMALIZED_organisationname'].apply(isinstance, args=(str,))]
df_spine = df_spine[df_spine['NORMALIZED_organisationname'].notnull()]
df_spine = df_spine.drop_duplicates(subset=['NORMALIZED_organisationname'], keep=False)
df_spine = df_spine[['uid', 'NORMALIZED_organisationname', 'organisationname', 'fulladdress', 'city', 'postcode', 'registerdate', 'removeddate']]
df_spine['organisationname'] = df_spine['organisationname'].replace('"', "[DQ]", regex=True)
df_spine['fulladdress'] = df_spine['fulladdress'].replace('"', "[DQ]", regex=True)
df_spine.to_csv(os.path.join('..', 'registers', 'spine_w_normalised.csv'), index=False)

  0%|          | 0/777409 [00:00<?, ?it/s]

### Section 1.3

Now the NHSSpend NHS register:

In [16]:
df_nhsdigital = pd.read_csv(os.path.join(
    '..', 'registers', 'nhs_uniq.csv')
                      )
df_nhsdigital['NORMALIZED_NHSDigital_Supplier'] = df_nhsdigital['NHSDigital_Supplier'].astype(str).progress_apply(normaliser)
df_nhsdigital = df_nhsdigital[df_nhsdigital['NORMALIZED_NHSDigital_Supplier'].apply(isinstance, args=(str,))]

df_nhsdigital = df_nhsdigital[df_nhsdigital['NORMALIZED_NHSDigital_Supplier'].notnull()]
df_nhsdigital = df_nhsdigital.drop_duplicates(subset=['NORMALIZED_NHSDigital_Supplier'], keep=False)
df_nhsdigital = df_nhsdigital[['NHSDigital_Supplier', 'NORMALIZED_NHSDigital_Supplier']]
df_nhsdigital['NHSDigital_Supplier'] = df_nhsdigital['NHSDigital_Supplier'].replace('"', "[DQ]", regex=True)
df_nhsdigital.to_csv(os.path.join('..', 'registers', 'nhsdigital_w_normalised.csv'), index=False)

  0%|          | 0/45739 [00:00<?, ?it/s]

# Section 2

### Section 2.1

Lets now load the files back in for the matching process.

#### Section 2.1.1

First, the uniq suppliers:

In [7]:
df_uniq = pd.read_csv(os.path.join(os.getcwd(),
                                   '..',
                                   'raw_data',
                                   'merged_groupby_raw.csv')
                     )
print(len(df_uniq))
df_uniq = df_uniq[df_uniq['NORMALIZED_SUPPLIER'].notnull()]
print(len(df_uniq))
print(f'We are then left with {len(df_uniq)} rows of unique "single" suppliers')

213632
213631
We are then left with 213631 rows of unique "single" suppliers


#### Section 2.1.2 

Then, the spine

In [8]:
df_spine = pd.read_csv(os.path.join('..', 'registers', 'spine_w_normalised.csv'), low_memory=False)

#### Section 2.1.3

Then, the CH:

In [19]:
df_ch = pd.read_csv(os.path.join('..', 'registers', 'ch_w_normalised.csv'))

#### Section 2.1.4

Then, the NHS Digital:

In [20]:
df_nhsdigital = pd.read_csv(os.path.join('..', 'registers', 'nhsdigital_w_normalised.csv'))

# Section 3

Do the matches here.
    
#### Section 3.1.

Make the spine results

In [13]:
df_spine

df_spine['removeddate'] = pd.to_datetime(df_spine['removeddate'], errors='coerce', dayfirst=True)

# Filter based on the condition
df_spine[~(df_spine['removeddate'] <= '2010-01-01')]

,uid,NORMALIZED_organisationname,organisationname,fulladdress,city,postcode,registerdate,removeddate
1,GB-CHC-200001,PAINTERSCOMPANY CHARITY,PAINTERS' COMPANY CHARITY,"PAINTERS' HALL, 9 LITTLE TRINITY LANE, LONDON",NaN,EC4V 2AD,08/06/1961,NaT
2,GB-CHC-200003,HERGA WORLD DISTRESS FUND,HERGA WORLD DISTRESS FUND,"5 HIGH STREET, HARROW ON THE HILL, MIDDLESEX",NaN,HA1 3HP,08/06/1961,NaT
3,GB-CHC-200004,WILLIAM GOLDSTEIN LAY STAFF BENEVOLENT FUND RO...,THE WILLIAM GOLDSTEIN LAY STAFF BENEVOLENT FUN...,"THE SPECIAL TRUSTEES OFFICE, 57B WEST SMITHFIE...",NaN,EC1A 9DS,08/06/1961,NaT
6,GB-CHC-200007,HOLDENHURST OLD PEOPLES HOME TRUST,THE HOLDENHURST OLD PEOPLE'S HOME TRUST,"BOUGH FARM, BURWASH WEALD, EAST SUSSEX",NaN,TN19 7LX,07/06/1967,NaT
8,GB-CHC-200009,RALPH LEVY CHARITABLE COMPANY LTD,THE RALPH LEVY CHARITABLE COMPANY LIMITED,"5-6 DOWN STREET, LONDON",NaN,W1J 7AH,17/03/1961,NaT
...,...,...,...,...,...,...,...,...
676316,GB-CHC-1163677,DARUL TACLIM WAL IFTA,DARUL TACLIM WAL IFTA,"106 HIGH ROAD, WILLESDEN, LONDON",NaN,NW10 2PP,21/09/2015,NaT
676317,GB-CHC-1179370,SAFE RESCUE LTD,SAFE RESCUE LTD,"11 LINCOLN AVENUE, HINGHAM, NORWICH",NaN,NR9 4NA,27/07/2018,NaT
676318,GB-COOP-R019720,WESTONSUPERMARE EDUCATION TRUST,Weston-super-Mare Education Trust,"OLDMIXON PRIMARY SCHOOL,MONKTON AVENUE",WESTON-SUPER-MARE,BS24 9DA,20/08/2013,2018-06-19
676319,GB-CHC-1185102,REFUGEE RIGHTS EUROPE LTD,REFUGEE RIGHTS EUROPE LTD,"71-75 SHELTON STREET, COVENT GARDEN, LONDON",NaN,WC2H 9JQ,29/08/2019,NaT


In [21]:
df_spine_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'],
                                df_spine['NORMALIZED_organisationname'],
                                'spine')
df_spine_results.to_csv(os.path.join('..', 'matches', 'matches_to_spine.csv'))

  0%|          | 0/213631 [00:00<?, ?it/s]

In [ ]:
df_spine_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'],
                                df_spine[~(df_spine['removeddate'] <= '2010-01-01')]['NORMALIZED_organisationname'],
                                'spine_notclosed2010')
df_spine_results.to_csv(os.path.join('..', 'matches', 'matches_to_spine_notclosed2010.csv'))

In [ ]:
df_spine_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'],
                                df_spine[(df_spine['removeddate'] <= '2010-01-01')]['NORMALIZED_organisationname'],
                                'spine_closed2010')
df_spine_results.to_csv(os.path.join('..', 'matches', 'matches_to_spine_closed2010.csv'))

  0%|          | 0/213631 [00:00<?, ?it/s]

In [22]:
df_uniq['NORMALIZED_SUPPLIER'][~df_uniq['NORMALIZED_SUPPLIER'].apply(isinstance, args=(str,))]

Series([], Name: NORMALIZED_SUPPLIER, dtype: object)

#### Section 3.2.

Make the CH results

In [23]:
df_ch_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'],
                             df_ch['NORMALIZED_CompanyName'],
                             'ch')
df_ch_results.to_csv(os.path.join('..', 'matches', 'matches_to_ch.csv'))

  0%|          | 0/213631 [00:00<?, ?it/s]

#### Section 3.3.

Make the NHS Digital results

In [24]:
df_nhsdigital_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'], df_nhsdigital['NORMALIZED_NHSDigital_Supplier'], 'nhsdigital')
df_nhsdigital_results.to_csv(os.path.join('..', 'matches', 'matches_to_nhsdigital.csv'))

  0%|          | 0/213631 [00:00<?, ?it/s]

# Section 4

Now, merge the matches back onto the unique.

### Section 4.1.

First, load the matches in

In [ ]:
df_spine_results = pd.read_csv(os.path.join('..', 'matches', 'matches_to_spine.csv'), index_col=0)
df_spine_results_notclosed2010 = pd.read_csv(os.path.join('..', 'matches', 'matches_to_spine_notclosed2010.csv'), index_col=0)
df_spine_results_closed2010 = pd.read_csv(os.path.join('..', 'matches', 'matches_to_spine_closed2010.csv.csv'), index_col=0)
df_ch_results = pd.read_csv(os.path.join('..', 'matches', 'matches_to_ch.csv'), index_col=0)
df_nhsdigital_results = pd.read_csv(os.path.join('..', 'matches', 'matches_to_nhsdigital.csv'), index_col=0)
df_uniq = pd.read_csv(os.path.join(os.getcwd(),
                                   '..',
                                   'raw_data',
                                   'merged_groupby_raw.csv')
                     )

### Section 4.2.

Add new empty fields

In [ ]:
df_uniq['verified_normalized_spine_name'] = np.nan
df_uniq['verified_spine_uid'] = np.nan
df_uniq['verified_normalized_ch_name'] = np.nan
df_uniq['verified_ch_uid'] = np.nan
df_uniq['verified_nhsdigital_name'] = np.nan

In [ ]:
df_uniq = df_uniq.join(df_spine_results, how='left')
df_uniq = df_uniq.join(df_spine_results_notclosed2010, how='left')
df_uniq = df_uniq.join(df_spine_results_closed2010, how='left')
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_with_approximate_spine.csv'),
               index=False)

In [ ]:
df_uniq = df_uniq.join(df_ch_results, how='left')
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_with_approximate_spine_and_ch.csv'),
               index=False)

In [ ]:
df_uniq = df_uniq.join(df_nhsdigital_results, how='left')
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_with_approximate_spine_and_ch_and_nhsdigital.csv'),
               index=False)

# Section 4

Do some EDA here which checks that the matches have been linked back in appropriately

In [ ]:
df_uniq = pd.read_csv(os.path.join(os.getcwd(),
                                   '..',
                                   'raw_data',
                                   'merged_groupby_with_approximate_spine_and_ch_and_nhsdigital_old.csv'),
                      index_col=None)

In [ ]:
df_uniq.columns

In [ ]:
# Fix strange indexing bug
df1 = df_uniq[['SUPPLIER', 'contractsfinder_awardedToVcse', 'contractsfinder_region',
               'date', 'dept', 'PAYMENT_TOTAL_AMOUNT', 'ORG_COUNT',
               'NORMALIZED_SUPPLIER', 'PAYMENT_TOTAL_COUNT', 'NHSSpend_CompanyNumber',
               'NHSSpend_CharityRegNo', 'NHSSpend_CharitySubNo', 'deptcount',
               'verified_normalized_spine_name', 'verified_spine_uid',
               'verified_normalized_ch_name', 'verified_ch_uid',
               'verified_nhsdigital_name']]
df1 = df1[df1['NORMALIZED_SUPPLIER'].notnull()]

df2 = df_uniq[['best_spine_match_1',
               'best_spine_match_1_score', 'best_spine_match_2',
               'best_spine_match_2_score', 'best_spine_match_3',
               'best_spine_match_3_score', 'best_spine_match_4',
               'best_spine_match_4_score', 'best_spine_match_5',
               'best_spine_match_5_score', 'best_ch_match_1', 'best_ch_match_1_score',
               'best_ch_match_2', 'best_ch_match_2_score', 'best_ch_match_3',
               'best_ch_match_3_score', 'best_ch_match_4', 'best_ch_match_4_score',
               'best_ch_match_5', 'best_ch_match_5_score', 'best_nhsdigital_match_1',
               'best_nhsdigital_match_1_score', 'best_nhsdigital_match_2',
               'best_nhsdigital_match_2_score', 'best_nhsdigital_match_3',
               'best_nhsdigital_match_3_score', 'best_nhsdigital_match_4',
               'best_nhsdigital_match_4_score', 'best_nhsdigital_match_5',
               'best_nhsdigital_match_5_score']]
df2 = df2[df2['best_spine_match_1_score'].notnull()]
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)
df_uniq = pd.concat([df1.copy(), df2.copy()], axis=1)

In [ ]:
df_spine_w_norm = pd.read_csv(os.path.join('..', 'registers', 'spine_w_normalised.csv'), index_col=None, low_memory=False)
df_spine_w_norm_names = df_spine_w_norm[['NORMALIZED_organisationname', 'organisationname']].drop_duplicates(subset=['NORMALIZED_organisationname'])
for match in ['1', '2', '3', '4', '5']:
    df_uniq = pd.merge(df_uniq, df_spine_w_norm_names, how='left', left_on='best_spine_match_' + str(match), right_on='NORMALIZED_organisationname')
    df_uniq = df_uniq.rename({'organisationname': 'best_spine_match_' + str(match) + '_notnormalised'}, axis=1)
    df_uniq = df_uniq.drop({'NORMALIZED_organisationname'}, axis=1)

In [ ]:
df_ch_w_norm = pd.read_csv(os.path.join('..', 'registers', 'ch_w_normalised.csv'), index_col=None, low_memory=False)
df_ch_w_norm_names = df_ch_w_norm[['CompanyName', 'NORMALIZED_CompanyName']].drop_duplicates(subset=['NORMALIZED_CompanyName'])
for match in ['1', '2', '3', '4', '5']:
    df_uniq = pd.merge(df_uniq, df_ch_w_norm_names, how='left', left_on='best_ch_match_' + str(match), right_on='NORMALIZED_CompanyName')
    df_uniq = df_uniq.rename({'CompanyName': 'best_ch_match_' + str(match) + '_notnormalised'}, axis=1)
    df_uniq = df_uniq.drop({'NORMALIZED_CompanyName'}, axis=1)

In [ ]:
df_nhsdigital_w_norm = pd.read_csv(os.path.join('..', 'registers', 'nhsdigital_w_normalised.csv'), index_col=None, low_memory=False)
df_nhsdigital_w_norm_names = df_nhsdigital_w_norm[['NHSDigital_Supplier', 'NORMALIZED_NHSDigital_Supplier']].drop_duplicates(subset=['NORMALIZED_NHSDigital_Supplier'])
for match in ['1', '2', '3', '4', '5']:
    df_uniq = pd.merge(df_uniq, df_nhsdigital_w_norm_names, how='left', left_on='best_nhsdigital_match_' + str(match), right_on='NORMALIZED_NHSDigital_Supplier')
    df_uniq = df_uniq.rename({'NHSDigital_Supplier': 'best_nhsdigital_match_' + str(match) + '_notnormalised'}, axis=1)
    df_uniq = df_uniq.drop({'NORMALIZED_NHSDigital_Supplier'}, axis=1)

In [ ]:
df_uniq = df_uniq[['SUPPLIER', 'contractsfinder_awardedToVcse', 'contractsfinder_region',
                   'date', 'dept', 'PAYMENT_TOTAL_AMOUNT', 'ORG_COUNT',
                   'NORMALIZED_SUPPLIER', 'PAYMENT_TOTAL_COUNT', 'NHSSpend_CompanyNumber',
                   'NHSSpend_CharityRegNo', 'NHSSpend_CharitySubNo', 'deptcount',
                   'verified_normalized_spine_name', 'verified_spine_uid',
                   'verified_normalized_ch_name', 'verified_ch_uid',
                   'verified_nhsdigital_name',
                   'best_spine_match_1',
                   'best_spine_match_1_score',
                   'best_spine_match_1_notnormalised',
                   'best_spine_match_2',
                   'best_spine_match_2_score',
                   'best_spine_match_2_notnormalised',
                   'best_spine_match_3',
                   'best_spine_match_3_score',
                   'best_spine_match_3_notnormalised',
                   'best_spine_match_4',
                   'best_spine_match_4_score',
                   'best_spine_match_4_notnormalised',
                   'best_spine_match_5',
                   'best_spine_match_5_score',
                   'best_spine_match_5_notnormalised',
                   'best_ch_match_1',
                   'best_ch_match_1_score',
                   'best_ch_match_1_notnormalised',
                   'best_ch_match_2',
                   'best_ch_match_2_score',
                   'best_ch_match_2_notnormalised',
                   'best_ch_match_3',
                   'best_ch_match_3_score',
                   'best_ch_match_3_notnormalised',
                   'best_ch_match_4',
                   'best_ch_match_4_score',
                   'best_ch_match_4_notnormalised',
                   'best_ch_match_5',
                   'best_ch_match_5_score',
                   'best_ch_match_5_notnormalised',
                   'best_nhsdigital_match_1',
                   'best_nhsdigital_match_1_score',
                   'best_nhsdigital_match_1_notnormalised',
                   'best_nhsdigital_match_2',
                   'best_nhsdigital_match_2_score',
                   'best_nhsdigital_match_2_notnormalised',
                   'best_nhsdigital_match_3',
                   'best_nhsdigital_match_3_score',
                   'best_nhsdigital_match_3_notnormalised',
                   'best_nhsdigital_match_4',
                   'best_nhsdigital_match_4_score',
                   'best_nhsdigital_match_4_notnormalised',
                   'best_nhsdigital_match_5',
                   'best_nhsdigital_match_5_score',
                   'best_nhsdigital_match_5_notnormalised']]
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_with_approximate_spine_and_ch_and_nhsdigital.csv'),
               index=False)

In [ ]:
df_uniq